## 团队版本1.0测试

In [1]:
# 2. Import libraries and modules
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
# 简单数据分层
from sklearn.model_selection import train_test_split
# 数据预处理
from sklearn import preprocessing
# 模型选择 随机森林回归
from sklearn import RandomForestRegressor
# 数据处理 pipeline 管道
from sklearn.pipeline import make_pipeline
# 模型参数自动调节
from sklearn.model_selection import GridSearchCV
# 计算拟合度与根均方误差
from sklearn.metrics import mean_squared_error, r2_score
# 模型保存
from sklearn.externals import joblib
 

/home/python/Envs/competitions/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/python/Envs/competitions/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/python/Envs/competitions/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# 3. Load red wine data.
# dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
# data = pd.read_csv(dataset_url, sep=';')

train_data = pd.read_csv('../data/public.train.csv')
test_data = pd.read_csv('../data/public.test.csv')
submit = pd.read_csv('../data/submit_example.csv') 


In [4]:
# 4. Split data into training and test sets
y = train_data['发电量']
X = train_data.drop(['ID','发电量'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                   )

In [5]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

hyperparameters = { 'randomforestregressor__max_features' : [ 'sqrt'],
                  'randomforestregressor__max_depth': [None]}


# 5. Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))
 
# 6. Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : [ 'sqrt'],
                  'randomforestregressor__max_depth': [None]}
 

In [6]:
# 7. Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
clf.fit(X_train, y_train)


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decr...mators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['sqrt'], 'randomforestregressor__max_depth': [None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [9]:
# 9. Evaluate model pipeline on test data
pred = clf.predict(X_test)
print (r2_score(y_test, pred))
print (mean_squared_error(y_test, pred))

0.9980366270247928
0.02373437447432535


In [10]:
# 10. Save model for future use
joblib.dump(clf, 'rf_regressor.pkl')
# To load: clf2 = joblib.load('rf_regressor.pkl')

['rf_regressor.pkl']

In [13]:
# 10. 将test_data的训练数据输入模型，计算出结果，并保存

clf = joblib.load("rf_regressor.pkl")

df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
test_feature = test_data.drop('ID', axis=1)
pre = clf.predict(test_feature)

df_result['Score'] = pre
df_result.to_csv('./submit/submit_version1.csv', index=False, header=False, float_format='%.8f')

In [15]:
print(1 / (1 + 0.02373437447432535))

0.9768158859699201


训练出来的模型与给出的测试集, 拟合度很高0.97~0.98 但是实际上只有0.84